## Collecte de données

Vous devez collecter et télécharger un ensemble d'images. Vous avez les tâches suivantes à programmer, en automatisant le processus autant que possible :

1.  Créer un dossier appelé *images*.
2.  Télécharger les images sous licence ouverte dans le dossier *images* (minimum 100
    images).
3.  Enregistrez les métadonnées de chaque image comme la taille de l'image, le format de l'image (.jpeg,
    .png, etc.), l'orientation de l'image (paysage, portrait, carré, etc.),
    date de création, modèle d'appareil photo, etc. dans un ou plusieurs fichiers JSON. Vous pouvez utiliser les informations [Exif](https://en.wikipedia.org/wiki/Exif) présentes dans les fichiers d'images.

In [ ]:
import os

# Spécifiez les chemins des dossiers que vous souhaitez créer
dossier_images = 'images'
dossier_data = 'data'

# Créez les dossiers
os.makedirs(dossier_images, exist_ok=True)
os.makedirs(dossier_data, exist_ok=True)

print(f"Le dossier '{dossier_images}' a été créé avec succès.")
print(f"Le dossier '{dossier_data}' a été créé avec succès.")

In [ ]:
! pip install SPARQLWrapper

In [ ]:
import os
import requests
from SPARQLWrapper import SPARQLWrapper, JSON
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor

# Initialiser le wrapper SPARQL
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
IMAGE_DIR = "images"  # Dossier de sauvegarde des images

# Définir la requête SPARQL pour obtenir des images de monuments
sparql.setQuery("""
SELECT ?monument ?monumentLabel ?image WHERE {
  ?monument wdt:P31 wd:Q839954.  # Monument
  ?monument wdt:P18 ?image.      # Image
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT 100
""")

sparql.setReturnFormat(JSON)

try:
    # Exécuter la requête et récupérer les résultats en JSON
    results = sparql.query().convert()
except Exception as e:
    print(f"❌ Erreur lors de l'exécution de la requête SPARQL : {e}")
    exit(1)

# Liste des images à télécharger avec des indices de 1 à 100
images = [(i + 1, result["image"]["value"]) for i, result in enumerate(results["results"]["bindings"])]

# Définition d'un User-Agent personnalisé
HEADERS = {
    "User-Agent": "MonScript/1.0 (mailto:mohamedguef@gmail.com)"  # Remplace par ton email pour respecter la politique
}

def download_image(data):
    index, image_url = data
    try:
        # Nom fixe sous le format image_1.jpg, image_2.jpg, ..., image_100.jpg
        image_name = f"image_{index}.jpg"
        image_path = os.path.join(IMAGE_DIR, image_name)

        # Télécharger l'image avec un User-Agent correct
        response = requests.get(image_url, headers=HEADERS, timeout=10)
        response.raise_for_status()  # Vérifie si la requête a réussi
        
        # Sauvegarder l'image
        with open(image_path, 'wb') as f:
            f.write(response.content)
        
        print(f"✅ Téléchargé : {image_name}")

    except requests.RequestException as e:
        print(f"⚠️ Erreur lors du téléchargement de {image_url} : {e}")
    except Exception as e:
        print(f"⚠️ Erreur inattendue : {e}")

# Téléchargement des images en parallèle
MAX_THREADS = 10  # Ajuste ce nombre en fonction de ta connexion et des performances

with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    executor.map(download_image, images)

print("✅ Tous les téléchargements sont terminés !")


In [ ]:
! pip install Pillow ExifRead

In [ ]:
! pip install tqdm

In [ ]:
import os
import json
from PIL import Image, UnidentifiedImageError
import exifread
from tqdm import tqdm  # Barre de progression

# Définir le dossier des images et le fichier de sortie JSON
IMAGE_DIR = "images"
DATA_DIR = "data"
METADATA_FILE = os.path.join(DATA_DIR, "metadata.json")

# Vérifier que le dossier "images" existe
if not os.path.exists(IMAGE_DIR):
    print(f"⚠️ Dossier '{IMAGE_DIR}' introuvable. Vérifiez que les images sont bien téléchargées.")
    exit(1)

# Liste pour stocker les métadonnées de toutes les images
metadata_list = []

# Liste des fichiers images triés
image_files = sorted([f for f in os.listdir(IMAGE_DIR) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])

# Parcourir les images et extraire les métadonnées
for image_name in tqdm(image_files, desc="📷 Extraction des métadonnées"):
    image_path = os.path.join(IMAGE_DIR, image_name)

    try:
        # Vérifier si l'image est corrompue en tentant de l'ouvrir
        with Image.open(image_path) as img:
            img.verify()  # Vérifie l'intégrité de l'image sans la charger
            img = Image.open(image_path)  # Recharge l'image pour la lecture des métadonnées
            width, height = img.size
            format_ = img.format
            orientation = 'Portrait' if height > width else 'Paysage' if width > height else 'Carré'

        # Vérifier si c'est un PNG (pas d'Exif)
        if format_.upper() == "PNG":
            exif_data = "Non disponible (format PNG)"
        else:
            # Extraire les métadonnées Exif avec exifread
            with open(image_path, 'rb') as f:
                tags = exifread.process_file(f, stop_tag="EXIF DateTimeOriginal", details=False)

            # Filtrer les champs problématiques
            exif_data = {
                "Date de creation": str(tags.get("EXIF DateTimeOriginal", "Inconnu")),
                "Modele appareil": str(tags.get("Image Model", "Inconnu")),
                "Marque appareil": str(tags.get("Image Make", "Inconnu")),
                "ISO": str(tags.get("EXIF ISOSpeedRatings", "Inconnu")),
                "Temps d'exposition": str(tags.get("EXIF ExposureTime", "Inconnu")),
                "Ouverture (f)": str(tags.get("EXIF FNumber", "Inconnu")),
                "Longueur focale": str(tags.get("EXIF FocalLength", "Inconnu")),
                "GPS Latitude": str(tags.get("GPS GPSLatitude", "Non disponible")),
                "GPS Longitude": str(tags.get("GPS GPSLongitude", "Non disponible")),
            }

            # Extraire l'Annee de creation de la date de création
            date_creation = exif_data["Date de creation"]
            if date_creation != "Inconnu":
                annee = date_creation.split(":")[0]  # La date est généralement au format "YYYY:MM:DD HH:MM:SS"
            else:
                annee = "Inconnu"

            # Ajouter l'Annee de creation au dictionnaire
            exif_data["Annee de creation"] = annee

            # Supprimer les champs corrompus (évite les erreurs "Possibly corrupted field")
            exif_data = {key: value for key, value in exif_data.items() if "Possibly corrupted" not in value}

        # Ajouter les métadonnées à la liste
        metadata_list.append({
            'Nom du fichier': image_name,
            'Taille': {'Largeur': width, 'Hauteur': height},
            'Format': format_,
            'Orientation': orientation,
            'Exif': exif_data
        })

    except UnidentifiedImageError:
        print(f"⚠️ Image illisible : {image_name} (corrompue ou format inconnu)")
    except Exception as e:
        print(f"⚠️ Erreur inattendue sur {image_name} : {e}")

# Enregistrer toutes les métadonnées dans un fichier JSON
try:
    with open(METADATA_FILE, 'w', encoding='utf-8') as f:
        json.dump(metadata_list, f, indent=4, ensure_ascii=False)
    print(f"✅ Métadonnées enregistrées dans '{METADATA_FILE}'")
except Exception as e:
    print(f"❌ Erreur lors de l'enregistrement du fichier JSON : {e}")


In [ ]:
! pip install opencv-python

In [ ]:
! pip install scikit-learn

In [ ]:
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans
import json

# Fonction pour extraire les couleurs dominantes
def get_dominant_colors(image_path, k=3, max_width=200):
    """
    Extrait les k couleurs dominantes d'une image en utilisant K-Means.
    :param image_path: Chemin de l'image
    :param k: Nombre de couleurs dominantes à extraire
    :param max_width: Largeur maximale de l'image redimensionnée
    :return: Liste des couleurs dominantes au format RGB
    """
    image = cv2.imread(image_path)
    if image is None:
        print(f"⚠️ Impossible de lire l'image : {image_path}")
        return None

    # Redimensionner l'image pour réduire le nombre de pixels
    height, width = image.shape[:2]
    if width > max_width:
        ratio = max_width / width
        new_size = (max_width, int(height * ratio))
        image = cv2.resize(image, new_size, interpolation=cv2.INTER_AREA)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convertir en RGB
    pixels = image.reshape(-1, 3)  # Redimensionner en tableau 2D (N pixels x 3 canaux)
    kmeans = KMeans(n_clusters=k, n_init=10)  # Appliquer K-Means
    kmeans.fit(pixels)
    colors = kmeans.cluster_centers_.astype(int)  # Extraire les couleurs dominantes
    return colors.tolist()  # Convertir en liste pour JSON

# Dossier contenant les images
IMAGE_DIR = "images"
DATA_DIR = "data"

# Vérifier que le dossier existe
if not os.path.exists(IMAGE_DIR):
    print(f"⚠️ Dossier '{IMAGE_DIR}' introuvable.")
    exit(1)

# Dictionnaire pour stocker les couleurs dominantes de chaque image
dominant_colors_data = {}

# Parcourir toutes les images du dossier
for image_name in os.listdir(IMAGE_DIR):
    if image_name.lower().endswith(('.jpg', '.jpeg', '.png')):  # Filtrer les fichiers images
        image_path = os.path.join(IMAGE_DIR, image_name)
        print(f"🔍 Traitement de l'image : {image_name}")

        # Extraire les couleurs dominantes
        dominant_colors = get_dominant_colors(image_path, k=3)  # Extraire 3 couleurs dominantes
        if dominant_colors:
            dominant_colors_data[image_name] = dominant_colors
            print(f"✅ Couleurs dominantes pour {image_name} : {dominant_colors}")
        else:
            print(f"⚠️ Aucune couleur dominante trouvée pour {image_name}")

# Enregistrer les résultats dans un fichier JSON
output_file = os.path.join(DATA_DIR, "dominant_colors.json")
try:
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(dominant_colors_data, f, indent=4, ensure_ascii=False)
    print(f"✅ Couleurs dominantes enregistrées dans '{output_file}'")
except Exception as e:
    print(f"❌ Erreur lors de l'enregistrement du fichier JSON : {e}")

In [ ]:
! pip install tensorflow

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import json
from PIL import UnidentifiedImageError

# Charger le modèle ResNet50 pré-entraîné
model = ResNet50(weights='imagenet')

Image.MAX_IMAGE_PIXELS = None  # Supprime la limite de pixels

def classify_image(image_path):
    try:
        # Charger et prétraiter l'image
        img = image.load_img(image_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        # Prédire les classes
        preds = model.predict(x)
        return decode_predictions(preds, top=3)[0]

    except UnidentifiedImageError:
        print(f"Erreur : Le fichier {image_path} n'est pas une image valide.")
        return []

# Liste pour stocker les prédictions de toutes les images
predictions_list = []

# Parcourir toutes les images dans le dossier 'images'
for file_name in os.listdir('images'):
    if file_name.startswith('image_') and file_name.endswith('.jpg'):
        image_path = os.path.join('images', file_name)

        # Classifier l'image
        predictions = classify_image(image_path)

        # Ajouter les prédictions à la liste si l'image est valide
        if predictions:
            predictions_list.append({
                'file_name': file_name,
                'predictions': [{'label': pred[1], 'score': float(pred[2])} for pred in predictions]
            })

        print(f"Prédictions pour {file_name}: {predictions}")

# Enregistrer toutes les prédictions dans un fichier JSON
with open('data/predictions.json', 'w') as f:
    json.dump(predictions_list, f, indent=4)

print("Prédictions enregistrées dans 'data/predictions.json'")


In [ ]:
! pip install tk

In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog, Label, Button, colorchooser, Listbox
from PIL import Image, ImageTk
import os
import json

IMAGE_DIR = "images"
DATA_DIR = "data"
USER_PREFERENCES_FILE = os.path.join(DATA_DIR, "user_preferences.json")
USER_TAGS_FILE = os.path.join(DATA_DIR, "user_tags.json")
USER_LIKES_FILE = os.path.join(DATA_DIR, "user_likes.json")

# Assurer que le dossier data existe
os.makedirs(DATA_DIR, exist_ok=True)

class MainMenu:
    def __init__(self, root, username):
        self.root = root
        self.root.title("Menu Principal")
        self.username = username

        self.label = Label(root, text=f"Bienvenue, {username}!")
        self.label.pack(pady=20)

        self.carousel_button = Button(root, text="Voir le Carrousel d'Images", command=self.open_carousel)
        self.carousel_button.pack(pady=10)

        self.preferences_button = Button(root, text="Gérer les Préférences", command=self.open_preferences)
        self.preferences_button.pack(pady=10)

        # Bind the Enter key to the return_to_menu method
        self.root.bind('<Return>', lambda event: self.return_to_menu())

    def open_carousel(self):
        self.root.destroy()
        root = tk.Tk()
        ImageCarousel(root, self.username, self.return_to_menu)
        root.mainloop()

    def open_preferences(self):
        self.root.destroy()
        root = tk.Tk()
        UserPreferences(root, self.username, self.return_to_menu)
        root.mainloop()

    def return_to_menu(self):
        root = tk.Tk()
        MainMenu(root, self.username)
        root.mainloop()

class ImageCarousel:
    def __init__(self, root, username, return_to_menu_callback):
        self.root = root
        self.root.title("Carrousel d'images")
        self.username = username
        self.return_to_menu_callback = return_to_menu_callback
        self.image_files = sorted([f for f in os.listdir(IMAGE_DIR) if f.startswith("image_") and f.endswith(".jpg")])
        self.index = 0
        self.likes = self.load_likes()
        self.tags = self.load_tags()

        self.image_label = Label(root)
        self.image_label.pack()

        self.prev_button = Button(root, text="⬅️", command=self.prev_image)
        self.prev_button.pack(side=tk.LEFT, padx=10)

        self.next_button = Button(root, text="➡️", command=self.next_image)
        self.next_button.pack(side=tk.RIGHT, padx=10)

        self.like_button = Button(root, text="❤️ Like", command=self.toggle_like, bg="white")
        self.like_button.pack(pady=5)

        self.tag_entry = tk.Entry(root)
        self.tag_entry.pack()
        self.tag_entry.bind('<Return>', lambda event: self.add_tag())
        self.tag_button = Button(root, text="Ajouter un Tag", command=self.add_tag)
        self.tag_button.pack(pady=5)

        self.view_tags_button = Button(root, text="Consulter Tags", command=self.view_tags)
        self.view_tags_button.pack(pady=5)

        self.back_button = Button(root, text="Retour au Menu", command=self.return_to_menu)
        self.back_button.pack(pady=10)

        # Bind the Enter key to the return_to_menu method
        self.root.bind('<Return>', lambda event: self.return_to_menu())

        self.load_image()
        self.update_like_button()

    def load_image(self):
        image_path = os.path.join(IMAGE_DIR, self.image_files[self.index])
        img = Image.open(image_path)
        img = img.resize((500, 500))
        self.photo = ImageTk.PhotoImage(img)
        self.image_label.config(image=self.photo)
        self.update_like_button()

    def next_image(self):
        self.index = (self.index + 1) % len(self.image_files)
        self.load_image()

    def prev_image(self):
        self.index = (self.index - 1) % len(self.image_files)
        self.load_image()

    def load_likes(self):
        if os.path.exists(USER_LIKES_FILE):
            with open(USER_LIKES_FILE, "r") as f:
                return json.load(f)
        return {}

    def save_likes(self):
        with open(USER_LIKES_FILE, "w") as f:
            json.dump(self.likes, f, indent=4)

    def toggle_like(self):
        current_image = self.image_files[self.index]
        if self.username not in self.likes:
            self.likes[self.username] = []

        if current_image in self.likes[self.username]:
            self.likes[self.username].remove(current_image)
            self.like_button.config(bg="white")
        else:
            self.likes[self.username].append(current_image)
            self.like_button.config(bg="red")

        self.save_likes()

    def update_like_button(self):
        current_image = self.image_files[self.index]
        if self.username in self.likes and current_image in self.likes[self.username]:
            self.like_button.config(bg="red")
        else:
            self.like_button.config(bg="white")

    def load_tags(self):
        if os.path.exists(USER_TAGS_FILE):
            with open(USER_TAGS_FILE, "r") as f:
                return json.load(f)
        return {}

    def save_tags(self):
        with open(USER_TAGS_FILE, "w") as f:
            json.dump(self.tags, f, indent=4)

    def add_tag(self):
        tag = self.tag_entry.get()
        current_image = self.image_files[self.index]
        if tag:
            if current_image not in self.tags:
                self.tags[current_image] = []
            self.tags[current_image].append(tag)
            self.save_tags()
            print(f"Tag ajouté pour {current_image}: {tag}")

    def view_tags(self):
        tag_window = tk.Toplevel(self.root)
        tag_window.title("Tags de l'image")

        current_image = self.image_files[self.index]
        ttk.Label(tag_window, text=f"Tags pour {current_image}:").pack()

        tag_listbox = Listbox(tag_window)
        tag_listbox.pack()

        if current_image in self.tags:
            for tag in self.tags[current_image]:
                tag_listbox.insert(tk.END, tag)

        def remove_tag():
            selected_tag = tag_listbox.get(tk.ACTIVE)
            if selected_tag in self.tags[current_image]:
                self.tags[current_image].remove(selected_tag)
                self.save_tags()
                tag_listbox.delete(tk.ACTIVE)
                print(f"Tag supprimé pour {current_image}: {selected_tag}")

        remove_button = Button(tag_window, text="Supprimer Tag", command=remove_tag)
        remove_button.pack()

    def return_to_menu(self):
        self.root.destroy()
        self.return_to_menu_callback()

class UserPreferences:
    def __init__(self, root, username, return_to_menu_callback):
        self.root = root
        self.root.title("Préférences Utilisateur")
        self.username = username
        self.return_to_menu_callback = return_to_menu_callback
        self.preferences = self.load_preferences()

        self.label = Label(root, text=f"Préférences de {username}:")
        self.label.pack(pady=20)

        self.color_button = Button(root, text="Choisir Couleur Préférée", command=self.choose_color)
        self.color_button.pack(pady=5)

        self.orientation_label = Label(root, text="Orientation Préférée:")
        self.orientation_label.pack(pady=5)
        self.orientation_var = tk.StringVar(value=self.preferences.get("orientation", "Paysage"))
        self.orientation_menu = ttk.OptionMenu(root, self.orientation_var, "Paysage", "Paysage", "Portrait")
        self.orientation_menu.pack(pady=5)

        self.size_label = Label(root, text="Taille d'Image Préférée:")
        self.size_label.pack(pady=5)
        self.size_var = tk.StringVar(value=self.preferences.get("size", "Moyenne"))
        self.size_menu = ttk.OptionMenu(root, self.size_var, "Moyenne", "Vignette", "Moyenne", "Grande")
        self.size_menu.pack(pady=5)

        self.tags_label = Label(root, text="Balises Favorites:")
        self.tags_label.pack(pady=5)
        self.tags_entry = tk.Entry(root)
        self.tags_entry.pack(pady=5)
        self.tags_entry.bind('<Return>', lambda event: self.add_favorite_tag())
        self.add_tag_button = Button(root, text="Ajouter une Balise", command=self.add_favorite_tag)
        self.add_tag_button.pack(pady=5)

        self.view_preferences_button = Button(root, text="Consulter Préférences", command=self.view_preferences)
        self.view_preferences_button.pack(pady=5)

        self.back_button = Button(root, text="Retour au Menu", command=self.return_to_menu)
        self.back_button.pack(pady=10)

        # Bind the Enter key to the return_to_menu method
        self.root.bind('<Return>', lambda event: self.return_to_menu())

    def load_preferences(self):
        if os.path.exists(USER_PREFERENCES_FILE):
            with open(USER_PREFERENCES_FILE, "r") as f:
                return json.load(f).get(self.username, {})
        return {}

    def save_preferences(self):
        all_preferences = {}
        if os.path.exists(USER_PREFERENCES_FILE):
            with open(USER_PREFERENCES_FILE, "r") as f:
                all_preferences = json.load(f)

        all_preferences[self.username] = {
            "color": self.preferences.get("color", "#ffffff"),
            "orientation": self.orientation_var.get(),
            "size": self.size_var.get(),
            "tags": self.preferences.get("tags", [])
        }

        with open(USER_PREFERENCES_FILE, "w") as f:
            json.dump(all_preferences, f, indent=4)

    def choose_color(self):
        color = colorchooser.askcolor()[1]
        if color:
            self.preferences["color"] = color
            self.save_preferences()
            print(f"Couleur préférée mise à jour: {color}")

    def add_favorite_tag(self):
        tag = self.tags_entry.get()
        if tag:
            if "tags" not in self.preferences:
                self.preferences["tags"] = []
            self.preferences["tags"].append(tag)
            self.save_preferences()
            print(f"Balise favorite ajoutée: {tag}")

    def view_preferences(self):
        preferences_window = tk.Toplevel(self.root)
        preferences_window.title("Préférences Utilisateur")

        ttk.Label(preferences_window, text=f"Préférences de {self.username}:").pack()

        preferences_text = f"Couleur: {self.preferences.get('color', 'Non définie')}\n"
        preferences_text += f"Orientation: {self.orientation_var.get()}\n"
        preferences_text += f"Taille: {self.size_var.get()}\n"
        preferences_text += f"Balises: {', '.join(self.preferences.get('tags', []))}"

        ttk.Label(preferences_window, text=preferences_text).pack()

    def return_to_menu(self):
        self.save_preferences()
        self.root.destroy()
        self.return_to_menu_callback()

class UserLogin:
    def __init__(self, root):
        self.root = root
        self.root.title("Connexion Utilisateur")

        self.username_label = Label(root, text="Nom d'utilisateur:")
        self.username_label.pack(pady=5)

        self.username_entry = tk.Entry(root)
        self.username_entry.pack(pady=5)
        self.username_entry.bind('<Return>', lambda event: self.login())

        self.login_button = Button(root, text="Se Connecter", command=self.login)
        self.login_button.pack(pady=20)

    def login(self):
        username = self.username_entry.get()
        if username:
            self.root.destroy()
            root = tk.Tk()
            MainMenu(root, username)
            root.mainloop()

if __name__ == "__main__":
    root = tk.Tk()
    UserLogin(root)
    root.mainloop()


In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import os

# Charger les couleurs dominantes
with open("./data/dominant_colors.json", "r") as f:
    dominant_colors = json.load(f)

# Charger les prédictions (tags)
with open("./data/predictions.json", "r") as f:
    predictions = json.load(f)

# Charger les métadonnées
with open("./data/metadata.json", "r") as f:
    metadata = json.load(f)

# Convertir les données en DataFrame
df_colors = pd.DataFrame(list(dominant_colors.items()), columns=["Nom du fichier", "Couleurs dominantes"])
df_predictions = pd.DataFrame(predictions)
df_metadata = pd.DataFrame(metadata)

# Vérification des colonnes avant fusion
print("Colonnes df_predictions:", df_predictions.columns)
print("Colonnes df_colors:", df_colors.columns)
print("Colonnes df_metadata:", df_metadata.columns)

# Vérifier l'existence de 'file_name' avant la fusion
if "file_name" in df_predictions.columns:
    df = pd.merge(df_predictions, df_colors, left_on="file_name", right_on="Nom du fichier")
    df = pd.merge(df, df_metadata, on="Nom du fichier")
else:
    raise ValueError("La colonne 'file_name' est absente du fichier predictions.json")

# Ajouter une colonne 'main_label' en gérant les cas où 'predictions' est vide ou mal formé
df["main_label"] = df["predictions"].apply(lambda x: x[0]["label"] if isinstance(x, list) and x else "Inconnu")

# Fonction pour déterminer la couleur dominante (Rouge, Vert, Bleu)
def get_dominant_color(colors):
    """
    Détermine la couleur dominante (Rouge, Vert, Bleu) à partir d'une liste de couleurs RGB.
    :param colors: Liste de couleurs RGB
    :return: "Rouge", "Vert", "Bleu" ou "Inconnu"
    """
    if not colors:
        return "Inconnu"
    
    # Prendre la première couleur dominante (la plus fréquente)
    dominant = colors[0]
    r, g, b = dominant
    
    # Déterminer quel canal est le plus élevé
    if r > g and r > b:
        return "Rouge"
    elif g > r and g > b:
        return "Vert"
    elif b > r and b > g:
        return "Bleu"
    else:
        return "Inconnu"  # Cas d'égalité ou indéterminé

# Appliquer la fonction pour déterminer la couleur dominante de chaque image
df_colors["Couleur dominante"] = df_colors["Couleurs dominantes"].apply(get_dominant_color)

# Compter le nombre d'images par couleur dominante
color_counts = df_colors["Couleur dominante"].value_counts()

# Filtrer pour ne garder que Rouge, Vert, Bleu
color_counts = color_counts[color_counts.index.isin(["Rouge", "Bleu", "Vert"])]




# Visualisation 1 : Nombre d'images par label
plt.figure(figsize=(10, 6))
sns.countplot(data=df, y="main_label", order=df["main_label"].value_counts().index, palette="viridis")
plt.title("Nombre d'images par label (tag principal)")
plt.xlabel("Nombre d'images")
plt.ylabel("Label")
plt.show()

# Visualisation : Nombre d'images par couleur dominante
plt.figure(figsize=(8, 5))
sns.barplot(x=color_counts.index, y=color_counts.values, palette=["red", "blue", "green"])
plt.title("Nombre d'images par couleur dominante")
plt.xlabel("Couleur dominante")
plt.ylabel("Nombre d'images")
plt.show()


# Visualisation 3 : Répartition des scores de prédiction
all_scores = [pred["score"] for sublist in df["predictions"] for pred in sublist]
plt.figure(figsize=(10, 6))
sns.histplot(all_scores, bins=20, kde=True, color="blue")
plt.title("Répartition des scores de prédiction")
plt.xlabel("Score de prédiction")
plt.ylabel("Fréquence")
plt.show()

# Visualisation 4 : Nombre d'images par orientation
if "Orientation" in df.columns:
    plt.figure(figsize=(8, 5))
    sns.countplot(data=df, x="Orientation", palette="magma")
    plt.title("Nombre d'images par orientation")
    plt.xlabel("Orientation")
    plt.ylabel("Nombre d'images")
    plt.show()
else:
    print("Aucune donnée sur l'orientation disponible.")

# Visualisation 5 : Modeles d'appareils photo les plus utilisés
df["Modele appareil"] = df["Exif"].apply(lambda x: x.get("Modele appareil", "Inconnu") if isinstance(x, dict) else "Inconnu")

plt.figure(figsize=(10, 6))
sns.countplot(data=df, y="Modele appareil", palette="plasma", order=df["Modele appareil"].value_counts().index)
plt.title("Modeles appareils photo les plus utilisés")
plt.xlabel("Nombre d'images")
plt.ylabel("Modele appareil")
plt.show()

# Extraire l'année de création
df["Annee de creation"] = df["Exif"].apply(lambda x: x.get("Annee de creation", "Inconnu") if isinstance(x, dict) else "Inconnu")

# Nettoyer les données
df_annee = df[df["Annee de creation"] != "Inconnu"]
df_annee = df_annee.dropna(subset=["Annee de creation"])
df_annee["Annee de creation"] = df_annee["Annee de creation"].astype(int)  # Convertir en entier
df_annee = df_annee.sort_values("Annee de creation")

# Visualisation : Nombre d'images par année de création
plt.figure(figsize=(12, 6))
sns.countplot(data=df_annee, x="Annee de creation", palette="coolwarm", order=sorted(df_annee["Annee de creation"].unique()))
plt.title("Nombre d'images par Année de création", fontsize=16)
plt.xlabel("Année de création", fontsize=14)
plt.ylabel("Nombre d'images", fontsize=14)
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()


# Fonctionnalité de profil utilisateur
def user_profile(image_names):
    """
    Affiche les informations relatives aux images préférées de l'utilisateur.
    :param image_names: Liste des noms des images préférées
    """
    user_images = df[df["Nom du fichier"].isin(image_names)]
    
    print("=== Informations pour les images sélectionnées ===")
    for _, row in user_images.iterrows():
        print(f"\nImage : {row['Nom du fichier']}")
        print(f"Label principal : {row['main_label']}")
        print(f"Score de prédiction : {row['predictions'][0]['score']:.2f}")
        print(f"Couleurs dominantes : {row['Couleurs dominantes']}")
        largeur = row['Taille'].get('Largeur', '?') if isinstance(row['Taille'], dict) else '?'
        hauteur = row['Taille'].get('Hauteur', '?') if isinstance(row['Taille'], dict) else '?'
        print(f"Taille : {largeur}x{hauteur}")
        print(f"Orientation : {row['Orientation'] if 'Orientation' in row else 'Inconnue'}")
        print(f"Modele appareil : {row['Modele appareil']}")  # Utilisation de guillemets simples à l'intérieur
        

# Exemple d'utilisation

# Récupérer tous les fichiers image du dossier
image_folder = "./images"
all_images = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
demo_images = ["image_1.jpg", "image_10.jpg", "image_100.jpg"]
user_profile(all_images)  # Afficher les informations pour toutes les images